In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict
from google.colab import files

In [8]:
df = pd.read_excel("SBA Loan data .xlsx")
# remove loans labeled Exempt and Canceled, and remove nan values
df = df[df.LoanStatus != "EXEMPT"]
df = df[df.LoanStatus != "CANCLD"]
df = df[df['LoanStatus'].notna()]
# drop useless columns
df = df.drop(columns=['Program', 'BorrName', 'BorrStreet','BorrCity','CDC_Name','CDC_Street','CDC_City','NaicsDescription','ProjectCounty','ThirdPartyLender_Name','ThirdPartyLender_City','ThirdPartyLender_State'])
# modify NaicsCode to only include the largest two-digit sector code
df['NaicsCode'] = df['NaicsCode'].apply(lambda x: str(x)[:2] if not pd.isnull(x) else "None")

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
'''
Feature engineering based on data already provided
'''
# Indicator: whether term is integer multiple of a year (1: Yes, 0: No)
df["Term Multiple"] =  df['TermInMonths'].apply(lambda x: 0 if int(x) % 12 != 0 else 1)

# Indicator: CDC state equals borrower state (1: equals, 0: not equal or missing)
df["Same State"] = (df['BorrState'] == df['CDC_State']).astype(int)
# drop CDC state
df = df.drop(columns=['CDC_State'])

# Indicator: Locate in CA (1: Yes, 0: No)
df['In CA'] = df['BorrState'].apply(lambda x: 0 if x != "CA" else 1)

# Indicator: There's 3rd party lender (1: Yes, 0: No)
df['ThirdPartyDollars'] = df['ThirdPartyDollars'].fillna(0) # if there's no third party, fill in 0 amount
df['Is ThirdParty'] = df['ThirdPartyDollars'].apply(lambda x: 0 if x == 0 else 1)

# Indicator: Interest rate is missing (1: Yes, 0: No)
df['Missing Interest'] = df['InitialInterestRate'].apply(lambda x: 0 if not pd.isnull(x) else 1)
# drop interest rate column
df = df.drop(columns = ['InitialInterestRate'])


# One-hot encoding (4 columns): Sec. 504 - Loan Guarantees - Private Sector Financed, Sec. 504 - Delta loans, funded 9/26/95, Sec. 504 - Premier Certified Lender Program, 504 Refinance
df = df.join(pd.get_dummies(df['subpgmdesc']))
# rename columns of one-hot encoding
df = df.rename(columns={'504 Refinance': 'Refinance', 'Sec. 504 - Delta loans, funded 9/26/95': 'Delta', 'Sec. 504 - Loan Guarantees - Private Sector Financed': 'Private Sector', 'Sec. 504 - Premier Certified Lender Program': 'Premier'})

# One-hot encode business types: replace nan with "MISSING"
df['BusinessType'] = df['BusinessType'].fillna("MISSING")
df = df.join(pd.get_dummies(df['BusinessType']))

# drop remaining columns that include nan or useless values
df = df.drop(columns = ['CDC_Zip'])


In [10]:
'''
Adds data beyond those that were already provided
'''

# process SP500 yearly return
df_SP500 = pd.read_csv('history.csv')
df_SP500 = df_SP500.rename(columns = {'2023': 'Year', '26.29': 'Return'})
yr, ret = np.array(df_SP500['Year']), np.array(df_SP500['Return'])
ret_dict = defaultdict(float)
for i,j in enumerate(yr):
  ret_dict[j] = ret[i]
# add return as a feature (year)
df["SP500 YR"] = df['ApprovalFiscalYear'].apply(lambda x: ret_dict[x])

# process USA unemployment rate
df_unemployment = pd.read_csv('us-national-unemployment-rate.csv', skiprows = 15)
df_unemployment['date'] = df_unemployment['date'].apply(lambda x: int(x[:4]))
yr, unemp = np.array(df_unemployment['date']), np.array(df_unemployment[' value'])
unemp_dict = defaultdict(float)
for i,j in enumerate(yr):
  unemp_dict[j] = unemp[i]
# add unemployment rate as feature (year)
df['Unemployment YR'] = df['ApprovalFiscalYear'].apply(lambda x: unemp_dict[x])

# process median home price for each year (average of all quarters)
df_home = pd.read_csv('MSPUS.csv')
df_home['DATE'] = df_home['DATE'].apply(lambda x: int(x[:4]))
yr, price = np.array(df_home['DATE']), np.array(df_home['MSPUS'])
home_dict = defaultdict(list)
for i,j in enumerate(yr):
  home_dict[j].append(price[i])
# add year-before average home price as feature (year)
df['Avg Home Price'] = df['ApprovalFiscalYear'].apply(lambda x: np.mean(home_dict[x]))

# process USA gdp growth rate annual change
df_gdp = pd.read_csv('united-states-gdp-growth-rate.csv', skiprows = 16)
df_gdp['date'] = df_gdp['date'].apply(lambda x: int(x[:4]))
yr, gdp = np.array(df_gdp['date']), np.array(df_gdp[' Annual Change'])
gdp_dict = defaultdict(float)
for i,j in enumerate(yr):
  gdp_dict[j] = gdp[i]
# add year-before gdp annual change as feature (year)
df['GDP Delta YR'] = df['ApprovalFiscalYear'].apply(lambda x: gdp_dict[x])

# drop the state with nan
df = df.dropna(subset = ["ProjectState"])

# get log(S&P500) price on the day of loan approval
df_SP500_Price = pd.read_csv('^GSPC.csv')
df_SP500_Price['Date'] = pd.to_datetime(df_SP500_Price['Date'], format='%Y-%m-%d')
df['Log S&P Open'] = df['ApprovalDate'].apply(lambda x: np.log(list(df_SP500_Price.loc[df_SP500_Price['Date'] >= x]['Open'])[0]))

# add state unemployment rate during the year of loan approval for borrower and project
df_state_unemp = pd.read_excel('Unemployment_State.xlsx')
df_state_unemp['DATE'] = df_state_unemp['DATE'].apply(lambda x: int(str(x)[:4]))
df_state_unemp = df_state_unemp.set_index('DATE')
yr,st_borr, st_proj = list(df['ApprovalFiscalYear']), list(df['BorrState']), list(df['ProjectState'])
borr_st_unemp, proj_st_unemp = [], []
for i,j in enumerate(yr):
  borr_st_unemp.append(df_state_unemp.loc[j,st_borr[i]])
  proj_st_unemp.append(df_state_unemp.loc[j,st_proj[i]])

df = df.reset_index(drop=True)
df['BorrState Unemployment'] = pd.Series(borr_st_unemp)
df['ProjectState Unemployment'] = pd.Series(proj_st_unemp)


# add personal income per capita in each state during the year of loan approval for borrower and project, as well as add missing indicators
df_income = pd.read_excel('Personal Income Per Capita by State.xlsx')
df_income = df_income.set_index('State')
yr,st_borr, st_proj = list(df['ApprovalFiscalYear']), list(df['BorrState']), list(df['ProjectState'])
borr_st_inc, proj_st_inc = [], []
for i,j in enumerate(yr):
  borr_st_inc.append(df_income.loc[st_borr[i], j])
  proj_st_inc.append(df_income.loc[st_proj[i], j])
df['BorrState Income'] = pd.Series(borr_st_inc)
df['ProjState Income'] = pd.Series(proj_st_inc)

df['Missing Borr Income'] = df['BorrState Income'].apply(lambda x: 0 if not pd.isnull(x) else 1)
df['Missing Proj Income'] = df['ProjState Income'].apply(lambda x: 0 if not pd.isnull(x) else 1)
df['BorrState Income'] = df['BorrState Income'].fillna(0)
df['ProjState Income'] = df['ProjState Income'].fillna(0)

# add GDP by State After 1998
df_stGDP = pd.read_excel('GDP by State After 1998.xlsx')
df_stGDP  = df_stGDP.set_index('State')
yr,st_borr, st_proj = list(df['ApprovalFiscalYear']), list(df['BorrState']), list(df['ProjectState'])
borr_st_gdp, proj_st_gdp = [], []
for i,j in enumerate(yr):
  borr_st_gdp.append(df_stGDP.loc[st_borr[i], j])
  proj_st_gdp.append(df_stGDP.loc[st_proj[i], j])
df['BorrState GDP'] = pd.Series(borr_st_gdp)
df['ProjState GDP'] = pd.Series(proj_st_gdp)

df['Missing Borr GDP'] = df['BorrState GDP'].apply(lambda x: 0 if not pd.isnull(x) else 1)
df['Missing Proj GDP'] = df['ProjState GDP'].apply(lambda x: 0 if not pd.isnull(x) else 1)
df['BorrState GDP'] = df['BorrState GDP'].fillna(0)
df['ProjState GDP'] = df['ProjState GDP'].fillna(0)


# add housing vacancy rate by state during the year
df_state_vac = pd.read_excel('Vacancy Rate.xlsx')
df_state_vac['Date'] = df_state_vac['Date'].apply(lambda x: int(str(x)[:4]))
df_state_vac = df_state_vac.set_index('Date')
yr,st_borr, st_proj = list(df['ApprovalFiscalYear']), list(df['BorrState']), list(df['ProjectState'])
borr_st_vac, proj_st_vac = [], []
for i,j in enumerate(yr):
  borr_st_vac.append(df_state_vac.loc[j,st_borr[i]])
  proj_st_vac.append(df_state_vac.loc[j,st_proj[i]])

df = df.reset_index(drop=True)
df['BorrState Vacancy'] = pd.Series(borr_st_vac)
df['ProjectState Vacancy'] = pd.Series(proj_st_vac)

In [12]:
# random split for train and test sets: 0.8-0.2 split
df_test = df.sample(frac=0.2, random_state=1)
df_train = df.drop(df_test.index)

df_test.to_csv('df_test.csv')
files.download('df_test.csv')

df_train.to_csv('df_train.csv')
files.download('df_train.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>